# Question 4: General Theory/Application

_No need to be verbose, it's not fun for anyone_

1. What part of S**L**A**M** did this project deal with? Why? What does the other part deal with and how would it generally work, given that you only have LIDAR scans, RGB video stream, and noisy pose data for a moving robot?


2. Loop closures play an important role in reducing drift, how would you go about detecting these?


3. Explain the structure of your Jacobian. Is the pose-graph fully connected? Why/Why not?


4. With what you know now, how would you describe and differentiate the SLAM frontend and backend? Why do we need to optimise our poses/map in the first place - where does the noise come from/why?

_Your Answer_

## 1
This project deals with the SLAM backend since we are implementing pose graph optimization in this project. In particular, we deal with the localization part of SLAM, since we are trying to estimate the robot's poses.
The SLAM frontend gives us noisy pose estimates along with odometry and loop closure constraints. The goal is to reduce this noise by posing SLAM as an optimization problem to improve our pose estimates, which is what we do in this project.

The SLAM frontend deals with obtaining sensor measurements, odometry, estimating loop closures, and estimating transforms to get a rough estimate of the robot's poses and motion.
- The LiDAR scanner gives us point clouds in each frame.
- Using ICP, we can estimate the transforms between the point clouds, hence estimating the motion of the robot.
- The noise is then reduced using the SLAM backend to obtain more accurate estimates.

## 2
Loop closure constraints help the robot identify places it has already visited previously. This helps it self-correct its path and hence reduce drift. Loop closures are detected by identifying data associations and feature correspondences between images. There are 2 approaches to identify loop closures:
- Map-to-map matching, which finds correspondences between common features in submaps generated in SLAM. Visual appearance of features and distances between features are used to identify common features. The submaps can then be aligned.
- Image-to-image matching, which finds correspondences between common features using RGB images. For example, using SIFT or SURF for feature mapping, bag of visual words.

#### Bag of visual words
Bag of visual words is an image-to-image mapping method for loop closure. In this method, an image is represented by a set of features, which consist of important points (keypoints) and descriptors which describe the keypoints. We use the keypoints and descriptors to construct vocabularies and represent each image as a frequency histogram of its set of features. From the frequency histogram, we can find similar images for loop closure.

## 3
The Jacobian structure is as follows:
- The number of rows is equal to the number of constraints in the optimization problem (odometry, loop closure and anchoring origin).
- The number of columns is the total number of poses we wish to optimize.
- In 1D slam, the constaints are all linear with respect to the poses. Hence the Jacobian is constant (derivative of a linear function of the variable yields a constant).

The pose graph is a sparse graph and is not fully connected. This is because of how we define the constraints. If there are n nodes:
- The odometry constraints connect only adjacent nodes. This results in (n-1) edges. If fully connected, there would be n(n-1) nodes.
- Loop closure constraints help the robot recognise a place it has seen before. Each landmark will be connected only to a certain number of other nodes (forming loops), not all. Further, the landmark nodes themselves are not connected to each other.
Hence there are a lot of edges missing from the graph and it is sparse.

The Jacobian shape for 2D SLAM turns out to be 420 x 360 = [3 * (1 + numOdo + numLoop)] x [3 * numPoses] where the number of rows explanation can be understood the same as for the residuals mentioned above. For the columns, numPoses is the number of poses/vertices (120) and we multiply by 3 as each pose has 3 parameters of x, y and theta which are to be optimized. Here the rows follow the same structure/order as the residuals mentioned above while the columns follow the order of the poses (from 0 - 119) with 3 consecutive columns per pose in the order of x<sub>i</sub>, y<sub>i</sub> and theta<sub>i</sub> (here i refers to pose index/number). Essentially, the Jacobian is calculated by taking the partial derivative of the residuals [3 * (1 + numOdo + numLoop)] with respect to all the parameters [3 * numPoses] and stacking them up.

## 4
The SLAM frontend deals with obtaining sensor measurements, odometry, estimating loop closures, and estimating transforms to get a rough estimate of the robot's poses and motion. The constraints obtained in the frontend give us a pose graph to optimize.
The SLAM backend deals with optimizing the noisy estimates obtained by the SLAM frontend by using pose graph optimization.

We need to optimize our robot's pose and map due to the noise in our initial estimates.
- Sensors are not 100% accurate and there will be some noise in the point clouds and odometry. 
- The correspondences used in ICP are not completely accurate, since they are just an estimate. This will result in noise in the transforms obtained between consecutive poses of the robot. Multiplying noisy transforms will further amplify the noise.
We use pose graph optimization to reduce this noise and improve our estimates.